# Pairwise trajectory distance Integral

This Jupyter Notebook demonstrates the implementation of the `integral_vec` function which calculates the integral of the distance between two points across a time interval using the trapezoidal rule. 

The integral represents the accumulated distance between two points `i` and `j` over time steps defined in `Fmap` and `timemap`. The function calculates pairwise distances at each time step and applies the trapezoidal rule to estimate the total integral.

The function also measures and prints the execution time for performance tracking.

### Steps in the Function:
1. **Pairwise Distance Calculation**: Compute the Euclidean distance between the points at consecutive time steps.
2. **Time Differences**: Compute the difference between consecutive time steps.
3. **Trapezoidal Rule**: The distances are multiplied by the time differences and summed to estimate the integral.
4. **Performance Timing**: The time taken to execute the function is printed.



Given the map $Fmap \in [m-1\times 2\times n]$, where $m$ is the number of timesteps and $n$ the number of trajectories. The second dimension inholds the x and y positions. We define the distance between each pair of trajectories $n_1$ and $n_2$ as follows

$$D(n1,n2)=\frac{1}{t_{m-1}-t_0}\sum_{k=0}^{m-2}\frac{t_{k+1}-t_k}{2}\Big[ dist\Big(F(k+1,:,n_2),F(k+1,:,n_1)\Big)+dist\Big(F(k,:,n_2),F(k,:,n_1)\Big)\Big]$$
We choose the Euclidian distance
$dist\Big(F(k,:,n_2),F(k,:,n_1)\Big)= \sqrt{\Big(F(k,x,n_2) - F(k,x,n_1)\Big)^2 + \Big(F(k,y,n_2) - F(k,y,n_1)\Big)^2}
$


Package to check for geodesics and great circles!! pyproj https://pyproj4.github.io/pyproj/stable/api/geod.html#pyproj.Geod

In [6]:
import numpy as np
import time
import sys, os
from pyproj import Geod

# get current directory
path = os.getcwd()
# get parent directory
parent_directory = os.path.sep.join(path.split(os.path.sep)[:-2])
# add utils and subfunctions folders to current working path
sys.path.append(parent_directory+"/subfunctions/latlon_transform")
# Import function for the polar rotation
from polar_rotation import polar_rotation_rx 




Given the map $Fmap \in [m-1\times 2\times n]$, where $m$ is the number of timesteps and $n$ the number of trajectories. The second dimension inholds the x and y positions. We define the distance between each pair of trajectories $n_1$ and $n_2$ as follows

$$D(n1,n2)=\frac{1}{t_{m-1}-t_0}\sum_{k=0}^{m-2}\frac{t_{k+1}-t_k}{2}\Big[ dist\Big(F(k+1,:,n_2),F(k+1,:,n_1)\Big)+dist\Big(F(k,:,n_2),F(k,:,n_1)\Big)\Big]$$
We choose the Euclidian distance
$dist\Big(F(k,:,n_2),F(k,:,n_1)\Big)= \sqrt{\Big(F(k,x,n_2) - F(k,x,n_1)\Big)^2 + \Big(F(k,y,n_2) - F(k,y,n_1)\Big)^2}
$


Package to check for geodesics and great circles!! pyproj https://pyproj4.github.io/pyproj/stable/api/geod.html#pyproj.Geod

In [5]:
def integral_vec(Fmap, timemap, i, j, geodesic = False):

    #start_time = time.time()  # Start the timer
    dT = timemap[-1]-timemap[0]

    if geodesic==False:
        # Compute pairwise distances at each time step
        distances = np.linalg.norm(Fmap[1:, :, i] - Fmap[1:, :, j], axis=1) + np.linalg.norm(Fmap[:-1, :, i] - Fmap[:-1, :, j], axis=1)
    else:
        # Define the WGS84 ellipsoid
        geod = Geod(ellps='WGS84')  #equivalent to +b=6356752 +a=6378137'

        #Go back to the non-rotated coordinates to compute the geodesic distances
        Fmap_lat,Fmap_lon = polar_rotation_rx(np.array(Fmap[:,1,:]),np.array(Fmap[:,0,:]),-90)  #Fmap[:,0,:] contains longitudes and Fmap[:,1,:] latitudes
        distances =  geod.inv(Fmap_lon[1:, i], Fmap_lat[1:, i], Fmap_lon[1:, j], Fmap_lat[1:, j])[2] + geod.inv(Fmap_lon[:-1, i], Fmap_lat[:-1, i], Fmap_lon[:-1, j], Fmap_lat[:-1, j])[2]  #distances in m
    
    # Compute time differences
    time_deltas = timemap[1:] - timemap[:-1]
    
    # Multiply by time and sum
    result = np.sum((time_deltas / 2) * distances)/dT
    
    #end_time = time.time()  # End the timer
    
    # Print the time taken
    #print(f"Execution time: {end_time - start_time:.6f} seconds")
    
    return result
